Training image-based models as baseline

In [1]:
%%bash
pip install attrdict
pip install timm
pip install pytorch-lightning==1.4.0

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
import os
import tqdm

import seaborn as sns
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision.utils import make_grid
from attrdict import AttrDict
import torch
import yaml
from sklearn.model_selection import StratifiedKFold
import copy
import pickle
# from tqdm import tqdm_notebook

# additional lightning 

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

import torch
from torch import nn
import torch.nn.functional as F
from timm import create_model

In [3]:
# import utility (this is for kaggle kernel)
# if you are in actual notebook use this -----
# import sys
# sys.path.append("../")
# from pawnet.utility import *


# if in kaggle
# you need to import the utility script by uploading to kaggle as pawnet_utility/pawnet_utility.py
from pawnet_utility import *

In [4]:
# check the package version to get reproducible env 
# source: https://www.kaggle.com/rtatman/get-the-versions-of-imported-packages

"""
To be used for kaggle notebook
"""

import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system names
        if name == "PIL":
            name = "Pillow"
        elif name == "sklearn":
            name = "scikit-learn"

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

In [5]:
# load config
# this object manages all the configurations

base_config_manager = BaseConfigLoader("../input/config/config.yaml")

# TODO: edit this for each base image model
model_config = base_config_manager.load_config().model.efficientnet_b0

In [6]:
model_config

# Loading Data

We will load the data by creating torch datasets as well as dataloader

In [7]:
# this is specific to kaggle
# if running in GCS, replace with our GCP bucket 
# get cache location of the dataset 
# GCS_DS_PATH = KaggleDatasets().get_gcs_path()
file_path = base_config_manager.load_config().filepath.kaggle #"/kaggle/input/petfinder-pawpularity-score/"


train_df = pd.read_csv(os.path.join(file_path,"train.csv"))
test_df = pd.read_csv(os.path.join(file_path,"test.csv"))

In [8]:
train_df.head()

In [9]:
test_df.head()

# Train basic models 

# Preparing for training - lightning variant 

* create relevant transformations, validation splits
* what this version differs is that we will be using the pytorch lightning framework - this allows easy TPU access for training 

lightning walkthrough: 
https://pytorch-lightning.readthedocs.io/en/latest/starter/introduction_guide.html
<br>
https://devblog.pytorchlightning.ai/train-anything-with-lightning-custom-loops-4be32314c961
<br>
https://github.com/PyTorchLightning/pytorch-lightning/blob/master/pl_examples/loop_examples/mnist_lite.py
<br>

In [10]:
"""
All pre-trained models expect input images normalized in the same way, 
i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), 
where H and W are expected to be at least 224. 
The images have to be loaded in to a range of [0, 1] and then 
normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]. 
You can use the following transform to normalize:

https://pytorch.org/vision/stable/models.html

"""
train_transformation = T.Compose(
            [
                T.Resize([224,224]),# imgnet needs at least 224
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ), # imgnet requirements 
            ]
        )
# train_data = pawnetDataset(annotation_df=train_df,img_dir = os.path.join(file_path,"train"),transform = train_transformation)
# # batchsize should be parameter in config
# train_loader = torch.utils.data.DataLoader(train_data,batch_size=64,num_workers =2, shuffle=True)


test_transformation = T.Compose([
                T.Resize([224,224]),# imgnet needs at least 224
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ), # imgnet requirements 
                ]
            )

# Training Loop

In [11]:
seed_everything(1)


# perfrom stratified sampling k fold model training
skf = StratifiedKFold(n_splits = model_config.n_splits, shuffle = True, random_state = 1)
splits = skf.split(train_df["Id"],train_df["Pawpularity"])


skf_train_list = [] # to store across folds
skf_valid_list = [] # to store across folds

for i, (train_index, test_index) in enumerate(splits):
    print("\n Starting: fold {}".format(i+1))
    
    # initialize model 
    criterion = torch.nn.BCEWithLogitsLoss()

    X_train, X_valid = train_df.iloc[train_index], train_df.iloc[test_index]
    X_train.reset_index(inplace=True,drop=True)
    X_valid.reset_index(inplace=True,drop=True)
    
    # build datamodule
    datamodule = PetfinderDataModule(img_dir = os.path.join(file_path,"train"),
                                     train_df=X_train,
                                     val_df=X_valid,
                                     train_transformation=train_transformation,
                                     test_transformation=test_transformation,
                                     batch_size=model_config.batch_size,
                                     num_workers=model_config.num_workers)
    
    
    model = pawNetBasic(criterion=criterion,model_config=model_config)
    print(model.summarize())
    earystopping = EarlyStopping(monitor="val_RMSE_loss",patience=5)
    lr_monitor = callbacks.LearningRateMonitor()
    loss_checkpoint = callbacks.ModelCheckpoint(
        filename="best_loss",
        monitor="val_RMSE_loss",
        save_top_k=1,
        mode="min",
        save_last=True,
    )
    logger = TensorBoardLogger(model_config.model_name)
    
    trainer = pl.Trainer(
        logger=logger, # tensorboard logger
        max_epochs=model_config.epoch,
        callbacks=[lr_monitor, loss_checkpoint, earystopping],
#          callbacks=[lr_monitor, loss_checkpoint],
         gpus=1,progress_bar_refresh_rate=1,accumulate_grad_batches=1
    )
    trainer.fit(model, datamodule=datamodule)

In [ ]:
# from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# # https://stackoverflow.com/questions/36700404/tensorflow-opening-log-data-written-by-summarywriter


# for i in range(5):
#     print(f"\n Fold: {i}==================================================")
#     path = [x for x in os.listdir(f"./pawnet_lightning_resnet/default/version_{i}/") if x.startswith("events")][0]
#     event_acc = EventAccumulator(os.path.join(f"pawnet_lightning_resnet/default/version_{i}/",path), size_guidance={'scalars': 0})
#     event_acc.Reload()

#     scalars = {}
#     for tag in event_acc.Tags()['scalars']:
#         events = event_acc.Scalars(tag)
#         scalars[tag] = [event.value for event in events]


#     print(scalars)

In [ ]:
# # this allow us to plot all metrics
# scalars

In [ ]:
# save
# with open("scalars.pkl","wb") as fout:
#     pickle.dump(scalars,fout)

# Load tensorboard (doesnt seem to work on kaggle) 

In [ ]:
# %load_ext tensorboard

# %tensorboard --logdir ./pawnet_lightning_resnet